In [2]:
%pip install langchain unstructured[all-docs] pydantic lxml openai chromadb tiktoken

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install unstructured[all-docs]

  Using cached unstructured_inference-0.7.36-py3-none-any.whl.metadata (5.9 kB)
  Using cached effdet-0.4.1-py3-none-any.whl.metadata (33 kB)
  Using cached google_cloud_vision-3.7.4-py2.py3-none-any.whl.metadata (5.2 kB)
  Using cached layoutparser-0.3.4-py3-none-any.whl.metadata (7.7 kB)
  Using cached timm-1.0.11-py3-none-any.whl.metadata (48 kB)
  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
Using cached unstructured_inference-0.7.36-py3-none-any.whl (56 kB)
Using cached effdet-0.4.1-py3-none-any.whl (112 kB)
Using cached google_cloud_vision-3.7.4-py2.py3-none-any.whl (467 kB)
Using cached timm-1.0.11-py3-none-any.whl (2.3 MB)
Using cached layoutparser-0.3.4-py3-none-any.whl (19.2 MB)
Using cached pdfplumber-0.11.4-py3-none-any.whl (59 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip list 

Package                  VersionNote: you may need to restart the kernel to use updated packages.

------------------------ -----------
aiohappyeyeballs         2.4.3
aiohttp                  3.10.10
aiosignal                1.3.1
annotated-types          0.7.0
antlr4-python3-runtime   4.9.3
anyio                    4.6.0
asgiref                  3.8.1
asttokens                2.4.1
attrs                    24.2.0
Authlib                  1.3.1
backoff                  2.2.1
bcrypt                   4.2.0
beautifulsoup4           4.12.3
build                    1.2.2.post1
cachetools               5.5.0
certifi                  2024.8.30
cffi                     1.17.1
chardet                  5.2.0
charset-normalizer       3.4.0
chroma-hnswlib           0.7.6
click                    8.1.7
colorama                 0.4.6
coloredlogs              15.0.1
comm                     0.2.2
contourpy                1.3.0
cryptography             43.0.1
cycler                   0.12.1
dataclass

## Data Loading

### Partition PDF tables, text, and images

* Use [Unstructured](https://unstructured-io.github.io/unstructured/) to partition elements

In [1]:
import os
input_path = os.getcwd()
output_path = os.path.join(os.getcwd(), "output")
print(input_path)
print(output_path)

c:\Users\khale\machine_learning\SMS\weaviate
c:\Users\khale\machine_learning\SMS\weaviate\output


In [3]:
import os

# Replace this path with the actual path to your Poppler bin directory
poppler_path = r'C:\Users\khale\Release-24.08.0-0\poppler-24.08.0\Library\bin'
os.environ['PATH'] += os.pathsep + poppler_path

In [4]:
print(os.environ['PATH'])

c:\Users\khale\machine_learning\SMS\weaviate\venv\Scripts;C:\Users\khale\machine_learning\SMS\weaviate\venv\Scripts;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Python311\Scripts\;C:\Python311\;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files\nodejs\;C:\ProgramData\chocolatey\bin;C:\Program Files\Git\cmd;C:\Program Files\Docker\Docker\resources\bin;C:\Program Files\PuTTY\;c:\Users\khale\AppData\Local\Programs\cursor\resources\app\bin;C:\Users\khale\.cargo\bin;C:\Users\khale\.console-ninja\.bin;C:\Users\khale\AppData\Local\Microsoft\WindowsApps;C:\Users\khale\AppData\Roaming\npm;C:\Users\khale\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\khale\AppData\Local\GitHubDesktop\bin;C:\Users\khale\AppData\Local\Programs\Ollama;C:\Users\khale\Release-24.08.0-0\poppler-24.08.0\Library\bin


In [5]:
# guide_mc.pdf
from unstructured.partition.pdf import partition_pdf
raw_pdf_elements = partition_pdf(
    filename=os.path.join(input_path, "guide_mc.pdf"),
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=output_path,
)



c:\Users\khale\machine_learning\SMS\weaviate\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# See all of the elements that Unstructured found,
titles = [elem for elem in raw_pdf_elements if elem.category == "Title"]

for title in titles:
    print(title.text)

In [ ]:
import textwrap

narrative_texts = [elem for elem in raw_pdf_elements if elem.category == "NarrativeText"]

for index, elem in enumerate(narrative_texts[:5]):
    print(f"Narrative text {index + 1}:")
    print("\n".join(textwrap.wrap(elem.text, width=70)))
    print("\n" + "-" * 70 + "\n")

In [ ]:
from typing import Any
import os
from unstructured.partition.pdf import partition_pdf
import pytesseract
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

input_path = os.getcwd()
output_path = os.path.join(os.getcwd(), "output")

# Get elements
raw_pdf_elements = partition_pdf(
    filename=os.path.join(input_path, "test.pdf"),
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=output_path,
)

In [ ]:
import base64

text_elements = []
table_elements = []
image_elements = []

# Function to encode images
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

for element in raw_pdf_elements:
    if 'CompositeElement' in str(type(element)):
        text_elements.append(element)
    elif 'Table' in str(type(element)):
        table_elements.append(element)

table_elements = [i.text for i in table_elements]
text_elements = [i.text for i in text_elements]

# Tables
print(len(table_elements))

# Text
print(len(text_elements))

In [ ]:
for image_file in os.listdir(output_path):
    if image_file.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(output_path, image_file)
        encoded_image = encode_image(image_path)
        image_elements.append(encoded_image)
print(len(image_elements))

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage

chain_gpt_35 = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024)
chain_gpt_4_vision = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)

# Function for text summaries
def summarize_text(text_element):
    prompt = f"Summarize the following text:\n\n{text_element}\n\nSummary:"
    response = chain_gpt_35.invoke([HumanMessage(content=prompt)])
    return response.content

# Function for table summaries
def summarize_table(table_element):
    prompt = f"Summarize the following table:\n\n{table_element}\n\nSummary:"
    response = chain_gpt_35.invoke([HumanMessage(content=prompt)])
    return response.content

# Function for image summaries
def summarize_image(encoded_image):
    prompt = [
        AIMessage(content="You are a bot that is good at analyzing images."),
        HumanMessage(content=[
            {"type": "text", "text": "Describe the contents of this image."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            },
        ])
    ]
    response = chain_gpt_4_vision.invoke(prompt)
    return response.content

In [ ]:
# Processing table elements with feedback and sleep
table_summaries = []
for i, te in enumerate(table_elements[0:2]):
    summary = summarize_table(te)
    table_summaries.append(summary)
    print(f"{i + 1}th element of tables processed.")

In [ ]:
# Processing text elements with feedback and sleep
text_summaries = []
for i, te in enumerate(text_elements[0:2]):
    summary = summarize_text(te)
    text_summaries.append(summary)
    print(f"{i + 1}th element of texts processed.")

In [ ]:
# Processing image elements with feedback and sleep
image_summaries = []
for i, ie in enumerate(image_elements[0:2]):
    summary = summarize_image(ie)
    image_summaries.append(summary)
    print(f"{i + 1}th element of images processed.")

## Multi-vector retriever

Use [multi-vector-retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary).

Summaries are used to retrieve raw tables and / or raw chunks of text.

### Add to vectorstore

Use [Multi Vector Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary) with summaries.

In [ ]:
import uuid

from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma

# Initialize the vector store and storage layer
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
store = InMemoryStore()
id_key = "doc_id"

# Initialize the retriever
retriever = MultiVectorRetriever(vectorstore=vectorstore, docstore=store, id_key=id_key)

# Function to add documents to the retriever
def add_documents_to_retriever(summaries, original_contents):
    doc_ids = [str(uuid.uuid4()) for _ in summaries]
    summary_docs = [
        Document(page_content=s, metadata={id_key: doc_ids[i]})
        for i, s in enumerate(summaries)
    ]
    retriever.vectorstore.add_documents(summary_docs)
    retriever.docstore.mset(list(zip(doc_ids, original_contents)))


In [ ]:
# Add text summaries
add_documents_to_retriever(text_summaries, text_elements)

# Add table summaries
add_documents_to_retriever(table_summaries, table_elements)

# Add image summaries
add_documents_to_retriever(image_summaries, image_summaries) # hopefully real images soon

# Table retrieval

The most complex table in the paper:

In [ ]:
# We can retrieve this table
retriever.get_relevant_documents(
    "What do you see on the images in the database?"
)

We can retrieve this image summary:

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

template = """Answer the question based only on the following context, which can include text, images and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke(
     "What do you see on the images in the database?"
)